In [233]:
import sys
sys.path.insert(0, '/usr/src/mytonctrl')

In [2]:
import requests as R
import mytonctrl as M


from datetime import datetime

# reset database

In [304]:
from backend.data.mongodb import get_database
db = get_database('ton_monitor_debug')
for collection in db.list_collection_names():
    db.drop_collection(collection)
    print(collection)

governance
ton_bridges
ton_apis
liteservers
network_stats


# TPS, validators, shards

In [249]:
last_block = ton.GetLastBlock()
last_block

'(-1,8000000000000000,16110318):B312495457F1E34A04133051350A91474E0C370117D5AF6E67CB69AF70BC1E60:AF44CE2552D016EEC342DB83AA3EA80815DF4F6DEB7B194C44EC74E1481465AD'

In [248]:
ton.GetTransactions('(-1,8000000000000000,16110296):6AF0B76A2FD2CF15F8FDF08E3BE927D38919DC89C76BB25AC4282EBA37AA7ADD:B5925025E14C70317D4FC41F3697A2C42A74FA93E18C971D41B4AD95EA4AF981')

[{'id': '1',
  'account': '3333333333333333333333333333333333333333333333333333333333333333',
  'lt': '23025745000001',
  'hash': 'B97BF57F4F4C8CC418D5AF1C6E3AD79A122D7814CA574285E262E290B110FF9F'},
 {'id': '2',
  'account': '3333333333333333333333333333333333333333333333333333333333333333',
  'lt': '23025745000002',
  'hash': '991B7074F812BAF8ACC9F09661AFBE58B376239B303E2B1CF8954100279843FA'},
 {'id': '3',
  'account': '34517C7BDF5187C55AF4F8B61FDC321588C7AB768DEE24B006DF29106458D7CF',
  'lt': '23025745000001',
  'hash': '802AC471ABA796C1294E6866B460462B2746E76694A227A9504BB21C2D61F501'},
 {'id': '4',
  'account': '34517C7BDF5187C55AF4F8B61FDC321588C7AB768DEE24B006DF29106458D7CF',
  'lt': '23025745000003',
  'hash': 'E0258CF2E56887FA2739D5399F4746C2B2D57996515700476F7AF8E8CDACBDAB'},
 {'id': '5',
  'account': '5555555555555555555555555555555555555555555555555555555555555555',
  'lt': '23025745000003',
  'hash': '575A67A4E3E15B57BA475F0CECB1F5B8CA505F89C539927010338FE56777770F'}]

In [246]:
ton.GetLastBlock()

'(-1,8000000000000000,16110296):6AF0B76A2FD2CF15F8FDF08E3BE927D38919DC89C76BB25AC4282EBA37AA7ADD:B5925025E14C70317D4FC41F3697A2C42A74FA93E18C971D41B4AD95EA4AF981'

In [15]:
ton = M.MyTonCore()


data = {}
data['tps'] = ton.GetStatistics('tpsAvg')[0]

# block rate
data['block_rate'] = {
    'masterchain': 3.0,
    'basechain': 2.0,
}

data['shardchains'] = ton.GetShardsNumber()

# validators
validators = ton.GetValidatorsList()
validators_total = len(validators)
validators_online = len(list(filter(lambda x: x['online'], validators)))
data['validators'] = {
    'online': validators_online,
    'total': validators_total
}

data

{'tps': 1.3,
 'block_rate': {'masterchain': 3.0, 'basechain': 2.0},
 'shardchains': 1,
 'validators': {'online': 124, 'total': 139}}

# dht servers

# check liteservers

In [234]:
# Считать список серверов
filePath = ton.liteClient.configPath
file = open(filePath, 'rt')
text = file.read()
file.close()
data = json.loads(text)

# Пройтись по серверам
result = list()
liteservers = data.get("liteservers")
for index, server in enumerate(liteservers):
    ip = int2ip(server['ip'])
    port = server['port']
    addr = f"{ip}:{port}"
    
    # working
    working = False
    try:
        ton.liteClient.Run("last", index=index)
        working = True
    except: pass
    
    # ping
    if working:
        runs = 10
        resp_time = measure_latency(ip, port=port, runs=runs, wait=0)
        ping = sum(resp_time) / runs
    else:
        ping = None
    
    # status
    try:
        resp = ton.liteClient.Run("last", index=index)
        if 'sync' in resp:
            print(resp)
    except: pass
    
    loc = {
        'addr': addr,
        'working': working,
        'response_time': ping,
        'syncronized': 'unknown',
    }
    print(loc)
    break
    result.append(loc)
#end for

# Записать данные в базу
result

{'addr': '67.207.74.182:4924', 'working': True, 'response_time': 47.36347198486328, 'syncronized': 'unknown'}


[]

# Services uptime

In [263]:
services_list = [{'name': 'ton.sh',
                  'api': 'https://api.ton.sh/getAddressInformation?address=EQCD39VS5jcptHL8vMjEXrzGaRcCVYto7HUn4bpAOg8xqB2N',
                  'ip': 'https://ton.sh'},
                 {'name': 'toncenter.com',
                  'api': 'https://toncenter.com/api/v2/getAddressInformation?address=0QCyt4ltzak71h6XkyK4ePfZCzJQDSVUNuvZ3VE7hP_Q-GTE',
                  'ip': 'https://toncenter.com'}]

services = []
for service in services_list:
    name = service['name']
    url = service['api']
    ip = server['ip']
    available = R.get(url, timeout=2).status_code == 200
    loc = {'name': name, 'available': available}
    services.append(loc)
services

TypeError: request() got an unexpected keyword argument 'timeouts'

# Governance

In [229]:
config17 = ton.GetConfig17()
config15 = ton.GetConfig15()
config34 = ton.GetConfig34()
config36 = ton.GetConfig36()

fullElectorAddr = ton.GetFullElectorAddr()


# get times
startWorkTime = ton.GetActiveElectionId(fullElectorAddr)
election_status = "open"
if startWorkTime == 0:
    startWorkTime = config36.get("startWorkTime")
    if startWorkTime is None:
        startWorkTime = config34.get("startWorkTime")
    election_status = "closed" 

electionsStartBefore = config15["electionsStartBefore"]
electionsEndBefore = config15["electionsEndBefore"]
validatorsElectedFor = config15["validatorsElectedFor"]

# compute election times
startElection = startWorkTime - electionsStartBefore
endElection = startWorkTime - electionsEndBefore
startNextElection = startElection + validatorsElectedFor

startElection = datetime.fromtimestamp(startElection)
endElection = datetime.fromtimestamp(endElection)
startNextElection = datetime.fromtimestamp(startNextElection)

elections = {
    'status': election_status,
    'start': startElection,
    'end': endElection,
    'start_next': startNextElection
}
elections

[debug]   14.11.2021, 22:14:25.374 (UTC)  <MainThread>  start GetConfig function (17)
[debug]   14.11.2021, 22:14:25.388 (UTC)  <MainThread>  start GetConfig function (15)
[debug]   14.11.2021, 22:14:25.401 (UTC)  <MainThread>  start GetConfig36 function
[debug]   14.11.2021, 22:14:25.417 (UTC)  <MainThread>  start GetFullElectorAddr function
[debug]   14.11.2021, 22:14:25.430 (UTC)  <MainThread>  start GetActiveElectionId function


{'status': 'closed',
 'start': datetime.datetime(2021, 11, 14, 14, 0, 39),
 'end': datetime.datetime(2021, 11, 14, 20, 50, 15),
 'start_next': datetime.datetime(2021, 11, 15, 8, 12, 55)}

In [228]:
offers = ton.GetOffersNumber()
offers_list = ton.GetOffers()
offers['offers_list'] = offers_list

offers

[debug]   14.11.2021, 22:14:22.412 (UTC)  <MainThread>  start GetOffersNumber function
[debug]   14.11.2021, 22:14:22.413 (UTC)  <MainThread>  start GetOffers function
[debug]   14.11.2021, 22:14:22.413 (UTC)  <MainThread>  start GetFullConfigAddr function
[debug]   14.11.2021, 22:14:22.463 (UTC)  <MainThread>  start GetConfig34 function
[debug]   14.11.2021, 22:14:22.479 (UTC)  <MainThread>  start GetOffers function


{'all': 0, 'new': 0, 'offers_list': []}

In [222]:
complaints = ton.GetComplaintsNumber()
complaints_list = ton.GetComplaints()
complaints['complaints_list'] = complaints_list

complaints

[debug]   14.11.2021, 22:08:26.077 (UTC)  <MainThread>  start GetComplaintsNumber function
[debug]   14.11.2021, 22:08:26.077 (UTC)  <MainThread>  start GetComplaints function
[debug]   14.11.2021, 22:08:26.077 (UTC)  <MainThread>  start GetFullElectorAddr function
[debug]   14.11.2021, 22:08:26.099 (UTC)  <MainThread>  start GetConfig32 function
[debug]   14.11.2021, 22:08:26.138 (UTC)  <MainThread>  start GetComplaints function


{'all': 0, 'new': 0, 'complaints_list': {}}

In [115]:
governance = {
    'elections': elections,
    'offers': offers,
    'complaints': complaints
}
governance

{'elections': {'status': 'open',
  'start': datetime.datetime(2021, 11, 14, 14, 0, 39),
  'end': datetime.datetime(2021, 11, 14, 20, 50, 15),
  'start_next': datetime.datetime(2021, 11, 15, 8, 12, 55)},
 'offers': {'all': 0, 'new': 0, 'offers_list': []},
 'complaints': {'all': 0, 'new': 0, 'complaints_list': {}}}

# Bridge check

In [165]:
from bs4 import BeautifulSoup


bridge_list = [{'name': 'ton-eth','url': 'https://ton.org/bridge/', 
                'address': 'Ef_dJMSh8riPi3BTUTtcxsWjG8RLKnLctNjAM4rw8NN-xWdr'},
               {'name': 'ton-bsc', 'url': 'https://ton.org/bridge/bsc', 
                'address': 'Ef9NXAIQs12t2qIZ-sRZ26D977H65Ol6DQeXc5_gUNaUys5r'},
               {'name': 'ton-other', 'url': 'https://ton.org/bridge/exmo', 
                'address': ''}]

bridges = []
for bridge_dict in bridge_list:
    resp = R.get(bridge_dict['url'])
    
    # web page status
    web_page_status = 'not available'
    if resp.status_code == 200:
        page = BeautifulSoup(resp.content, features="html.parser")
        button = page.find_all(attrs={'id': ['transferButton']})
        if len(button) > 0:
            web_page_status = 'available'
            
    # api status
    smart_contract_state = None
    if bridge_dict['address']:
        cmd = 'runmethod {address} get_bridge_data'.format(**bridge_dict)
        resp = ton.liteClient.Run(cmd)
        for row in resp.split('\n'):
            if 'result:' in row and '[' in row:
                row = row[row.find('[')+1:row.find(']')].strip()
                row = row.split()
                if len(row) > 0:
                    smart_contract_state = int(row[0])
    bridges.append({
        'name': bridge_dict['name'],
        'url': bridge_dict['url'],
        'web_page_status': web_page_status,
        'smart_contract_state': smart_contract_state
    })
bridges

[{'name': 'ton-eth',
  'url': 'https://ton.org/bridge/',
  'web_page_status': 'available',
  'smart_contract_state': 0},
 {'name': 'ton-bsc',
  'url': 'https://ton.org/bridge/bsc',
  'web_page_status': 'available',
  'smart_contract_state': 0},
 {'name': 'ton-other',
  'url': 'https://ton.org/bridge/exmo',
  'web_page_status': 'not available',
  'smart_contract_state': None}]

# Backup

# OLD

In [ ]:
lite = M.LiteClient()

In [ ]:
ton.GetConfig12()

In [ ]:
ton.GetConfig15()

In [ ]:
config36 = ton.GetConfig36()
config36

In [ ]:
shardsNumber = ton.GetShardsNumber()
print('shards_number:', shardsNumber)

In [ ]:
online_validators = ton.GetOnlineValidators()
validator_efficiency = ton.GetValidatorEfficiency()

In [ ]:
config15 = ton.GetConfig15()
config17 = ton.GetConfig17()

In [ ]:
statistics = ton.GetSettings("statistics")
statistics

In [ ]:
rootWorkchainEnabledTime_int = ton.GetRootWorkchainEnabledTime()
rootWorkchainEnabledTime_int

In [ ]:
validators_info = ton.GetValidatorsLoad()
validators_info[0]

In [ ]:
compliants_info = ton.GetComplaints()
compliants_info

In [ ]:
ton.GetAdnlAddr()

In [ ]:
statistics = ton.GetSettings("statistics")
tpsAvg = ton.GetStatistics("tpsAvg", statistics)
netLoadAvg = ton.GetStatistics("netLoadAvg", statistics)
disksLoadAvg = ton.GetStatistics("disksLoadAvg", statistics)
disksLoadPercentAvg = ton.GetStatistics("disksLoadPercentAvg", statistics)

statistics

In [ ]:
def PrintStatus(args):
    opt = None
    if len(args) == 1:
        opt = args[0]
    rootWorkchainEnabledTime_int = ton.GetRootWorkchainEnabledTime()
    config34 = ton.GetConfig34()
    config36 = ton.GetConfig36()
    totalValidators = config34["totalValidators"]
    onlineValidators = None
    validatorEfficiency = None
    if opt != "fast":
        onlineValidators = ton.GetOnlineValidators()
        validatorEfficiency = ton.GetValidatorEfficiency()
    if onlineValidators:
        onlineValidators = len(onlineValidators)
    oldStartWorkTime = config36.get("startWorkTime")
    if oldStartWorkTime is None:
        oldStartWorkTime = config34.get("startWorkTime")
    shardsNumber = ton.GetShardsNumber()
    validatorStatus = ton.GetValidatorStatus()
    config15 = ton.GetConfig15()
    config17 = ton.GetConfig17()
    fullConfigAddr = ton.GetFullConfigAddr()
    fullElectorAddr = ton.GetFullElectorAddr()
    startWorkTime = ton.GetActiveElectionId(fullElectorAddr)
    validatorIndex = ton.GetValidatorIndex()
    validatorWallet = ton.GetLocalWallet(ton.validatorWalletName)
    dbSize = ton.GetDbSize()
    offersNumber = ton.GetOffersNumber()
    complaintsNumber = ton.GetComplaintsNumber()
    statistics = ton.GetSettings("statistics")
    tpsAvg = ton.GetStatistics("tpsAvg", statistics)
    netLoadAvg = ton.GetStatistics("netLoadAvg", statistics)
    disksLoadAvg = ton.GetStatistics("disksLoadAvg", statistics)
    disksLoadPercentAvg = ton.GetStatistics("disksLoadPercentAvg", statistics)
    if validatorWallet is not None:
        validatorAccount = ton.GetAccount(validatorWallet.addr)
    else:
        validatorAccount = None
    PrintTonStatus(startWorkTime, totalValidators, onlineValidators, shardsNumber, offersNumber, complaintsNumber, tpsAvg)
    PrintLocalStatus(validatorIndex, validatorEfficiency, validatorWallet, validatorAccount, validatorStatus, dbSize, netLoadAvg, disksLoadAvg, disksLoadPercentAvg)
    PrintTonConfig(fullConfigAddr, fullElectorAddr, config15, config17)
    PrintTimes(rootWorkchainEnabledTime_int, startWorkTime, oldStartWorkTime, config15)